# Brewer Beer Style Classification

---

## Workflow stages

The competition solution workflow goes through seven stages described in the Data Science Solutions book.

1. Question or problem definition.
2. Acquire training and testing data.
3. Wrangle, prepare, cleanse the data.
4. Analyze, identify patterns, and explore the data.
5. Model, predict and solve the problem.
6. Visualize, report, and present the problem solving steps and final solution.
7. Supply or submit the results.

The workflow indicates general sequence of how each stage may follow the other. However there are use cases with exceptions.

- We may combine mulitple workflow stages. We may analyze by visualizing data.
- Perform a stage earlier than indicated. We may analyze data before and after wrangling.
- Perform a stage multiple times in our workflow. Visualize stage may be used multiple times.
- Drop a stage altogether. We may not need supply stage to productize or service enable our dataset for a competition.


## Question and problem definition

> For this competition, we have provided a dataset with 93 features for more than 200,000 products. The objective is to build a predictive model which is able to distinguish between our main product categories.

## Workflow goals

The data science solutions workflow solves for seven major goals.

**Classifying.** We may want to classify or categorize our samples. We may also want to understand the implications or correlation of different classes with our solution goal.

**Correlating.** One can approach the problem based on available features within the training dataset. Which features within the dataset contribute significantly to our solution goal? Statistically speaking is there a [correlation](https://en.wikiversity.org/wiki/Correlation) among a feature and solution goal? As the feature values change does the solution state change as well, and visa-versa? This can be tested both for numerical and categorical features in the given dataset. We may also want to determine correlation among features other than survival for subsequent goals and workflow stages. Correlating certain features may help in creating, completing, or correcting features.

**Converting.** For modeling stage, one needs to prepare the data. Depending on the choice of model algorithm one may require all features to be converted to numerical equivalent values. So for instance converting text categorical values to numeric values.

**Completing.** Data preparation may also require us to estimate any missing values within a feature. Model algorithms may work best when there are no missing values.

**Correcting.** We may also analyze the given training dataset for errors or possibly innacurate values within features and try to corrent these values or exclude the samples containing the errors. One way to do this is to detect any outliers among our samples or features. We may also completely discard a feature if it is not contribting to the analysis or may significantly skew the results.

**Creating.** Can we create new features based on an existing feature or a set of features, such that the new feature follows the correlation, conversion, completeness goals.

**Charting.** How to select the right visualization plots and charts depending on nature of the data and the solution goals.

In [1]:
# disables warnings (for local running with jupyter)
import warnings;
warnings.simplefilter('ignore')

# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## Acquire data

The Python Pandas packages helps us work with our datasets. We start by acquiring the training and testing datasets into Pandas DataFrames. We also combine these datasets to run certain operations on both datasets together.

In [2]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
test_check_df = pd.read_csv('../input/check-test.csv')
combine = [train_df, test_df]

## Analyze by describing data

Pandas also helps describe the datasets answering following questions early in our project.

**Which features are available in the dataset?**

In [3]:
print(train_df.columns.values)

['BeerID' 'StyleID' 'Size(L)' 'OG' 'FG' 'ABV' 'IBU' 'Color' 'BoilSize'
 'BoilTime' 'BoilGravity' 'Efficiency' 'MashThickness' 'SugarScale'
 'BrewMethod' 'PitchRate' 'PrimaryTemp' 'PrimingMethod' 'PrimingAmount']


**Which features are categorical?**

These values classify the samples into sets of similar samples. Within categorical features are the values nominal, ordinal, ratio, or interval based? Among other things this helps us select the appropriate plots for visualization.

- **Categorical**: `SugarScale`, `BrewMethod`, `PrimingMethod`
- **Ordinal**: None

**Which features are numerical?**

Which features are numerical? These values change from sample to sample. Within numerical features are the values discrete, continuous, or timeseries based? Among other things this helps us select the appropriate plots for visualization.

- **Continous**: `Size(L)`, `OG`, `FG`, `ABV`, `IBU`, `Color`, `BoilSize`, `BoilGravity`, `Efficiency`, `MashThickness`, `PitchRate`, `PrimaryTemp`, `PrimingAmount`
- **Discrete**: None

In [4]:
# preview the data
train_df.head()

,BeerID,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount
0,18751,102,20.82,1.058,1.018,5.29,19.44,9.24,29.34,90,1.041,75.0,1.25,Specific Gravity,All Grain,NaN,12.78,NaN,NaN
1,54259,7,44.00,1.068,1.016,6.81,60.20,10.08,54.00,60,1.056,68.0,2.50,Specific Gravity,All Grain,NaN,NaN,NaN,NaN
2,49642,115,37.85,1.055,1.013,5.50,26.25,7.97,45.42,90,1.046,70.0,1.60,Specific Gravity,All Grain,1.50,11.67,kegged/co2,2.5vol.@38deg. 9.1psi
3,61229,7,18.93,1.059,1.011,6.29,57.71,7.54,11.36,60,1.099,35.0,NaN,Specific Gravity,extract,0.35,24.44,NaN,NaN
4,5053,132,20.82,1.087,1.020,9.40,95.30,35.41,26.50,60,1.068,73.0,1.33,Specific Gravity,All Grain,0.75,18.33,NaN,NaN


**Which features are mixed data types?**

Numerical, alphanumeric data within same feature. These are candidates for correcting goal.

- `SugarScale`, `PrimingAmount`, `OG`, `FG`, `BoilGravity`

**Which features may contain errors or typos?**

This is harder to review for a large dataset, however reviewing a few samples from a smaller dataset may just tell us outright, which features may require correcting.

- `OG` > 1.150 SG / 34.3 Brix
- `FG` > 1.150 SG / 34.3 Brix
- `FG` < 1.005 SG / 1.3 Brix
- `BoilGravity` > 1.150 SG / 34.3 Brix
- `IBU` > 150
- `ABV` > 15
- `Color` > 50

In [5]:
train_df.tail()

,BeerID,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount
29539,65565,10,21.77,1.066,1.016,6.57,120.11,16.08,28.01,60,1.051,75.0,1.375,Specific Gravity,All Grain,0.75,18.33,Corn Sugar,4.3 oz.
29540,41574,10,20.82,1.064,1.012,6.80,39.11,7.19,13.25,60,1.101,35.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN
29541,67580,7,20.82,1.050,1.009,5.36,40.29,5.84,28.39,60,1.037,72.0,1.500,Specific Gravity,All Grain,1.00,18.33,NaN,NaN
29542,64816,9,20.00,1.055,1.009,6.03,54.05,4.44,25.00,60,1.044,70.0,3.000,Specific Gravity,All Grain,0.35,18.00,NaN,NaN
29543,18119,10,18.00,1.046,1.011,4.69,10.02,3.86,25.75,65,1.032,65.0,4.530,Specific Gravity,All Grain,0.35,25.00,Sugar,121 g


**Which features contain blank, null or empty values?**

These will require correcting.

- `BoilGravity`, `MashThickness`, `PitchRate`, `PrimaryTemp`, `PrimingMethod`, `PrimingAmount`

**What are the data types for various features?**

Helping us during converting goal.

- 3 features are integer. 2 in case of test datase
- 12 features are float.
- 4 features are string (object).

In [6]:
train_df.info()
print('_'*40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29544 entries, 0 to 29543
Data columns (total 19 columns):
BeerID           29544 non-null int64
StyleID          29544 non-null int64
Size(L)          29544 non-null float64
OG               29544 non-null float64
FG               29544 non-null float64
ABV              29544 non-null float64
IBU              29544 non-null float64
Color            29544 non-null float64
BoilSize         29544 non-null float64
BoilTime         29544 non-null int64
BoilGravity      28353 non-null float64
Efficiency       29544 non-null float64
MashThickness    17615 non-null float64
SugarScale       29544 non-null object
BrewMethod       29544 non-null object
PitchRate        13952 non-null float64
PrimaryTemp      20562 non-null float64
PrimingMethod    2766 non-null object
PrimingAmount    1973 non-null object
dtypes: float64(12), int64(3), object(4)
memory usage: 4.3+ MB
________________________________________
<class 'pandas.core.frame.DataFrame'>
R

# **What is the distribution of numerical feature values across the samples?**

This helps us determine, among other early insights, how representative is the training dataset of the actual problem domain.

- Total samples are **29,544** or **40%** of the actual number of beers registered in the dataset (**73,860**).
- `StyleId` is a categorical feature with values from **1** to **176**.
- Most beers (**95%**) have an `OG` lower than **1.100** SG.
- Most beers (**88%**) have a `FG` lower than **1.020** SG.
- Most beers (**89%**) have an `ABV` lower than **8**.
- Most beers (**93%**) have an `IBU` lower than **100**.
- Most beers (**84%**) have a `Color` lower than **25**.
- Most recipes (**> 75%**) have a `Size(L)` batch lower than **24** Liters.
- Most recipes (**> 75%**) have a `BoilSize` lower than **30** Liters.
- Most recipes (**99%**) have a `BoilTime` lower than **90** minutes.
- Most beers (**> 75%**) have an `BoilGravity` lower than **1.060** SG.
- Most beers (**87%**) have an `Efficiency` lower than **75%**.


In [7]:
train_df.describe()

,BeerID,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,PitchRate,PrimaryTemp
count,29544.000000,29544.000000,29544.000000,29544.000000,29544.000000,29544.000000,29544.000000,29544.000000,29544.000000,29544.000000,28353.000000,29544.000000,17615.000000,13952.000000,20562.000000
mean,37064.822705,60.265807,43.858097,1.398985,1.073568,6.137766,44.522199,13.465916,49.613009,65.165110,1.349680,66.251412,2.138660,0.751770,19.144404
std,21304.598971,56.864997,181.549585,2.170256,0.418615,1.942791,48.952740,11.956743,193.881710,15.022197,1.923937,14.162816,1.775947,0.397708,4.219790
min,3.000000,1.000000,1.000000,1.000000,-0.003000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-17.780000
25%,18589.250000,10.000000,18.930000,1.051000,1.011000,5.070000,23.380000,5.190000,20.820000,60.000000,1.040000,65.000000,1.500000,0.350000,18.000000
50%,37235.000000,35.000000,20.820000,1.058000,1.013000,5.790000,35.870000,8.470000,27.440000,60.000000,1.047000,70.000000,1.500000,0.750000,20.000000
75%,55479.250000,111.000000,23.660000,1.068000,1.017000,6.830000,56.330000,16.892500,30.000000,60.000000,1.060000,75.000000,3.000000,1.000000,20.560000
max,73858.000000,176.000000,9200.000000,34.034500,10.341400,53.810000,3409.300000,108.650000,9700.000000,240.000000,52.600000,100.000000,63.000000,2.000000,114.000000


In [8]:
train_df['OG'][train_df['OG'] <= 1.15][train_df['OG'] >= 1.005].count() / train_df['OG'].count()

0.9713985919306797

In [9]:
train_df['FG'][train_df['FG'] <= 1.05][train_df['FG'] >= 1.001].count() / train_df['FG'].count()

0.9717709179528838

In [10]:
train_df['ABV'][train_df['ABV'] <= 20][train_df['ABV'] >= 0.1].count() / train_df['ABV'].count()

0.9985106959111834

In [11]:
train_df['IBU'][train_df['IBU'] <= 150][train_df['IBU'] >= 0.1].count() / train_df['IBU'].count()

0.9285472515569997

In [12]:
train_df['Color'][train_df['Color'] <= 45][train_df['Color'] >= 0].count() / train_df['Color'].count()

0.9686907663146493

In [13]:
train_df['BoilGravity'][train_df['BoilGravity'] <= 1.15][train_df['BoilGravity'] >= 1.005].count() / train_df['BoilGravity'].count()

0.9609212429019857

In [14]:
train_df['Efficiency'][train_df['Efficiency'] <= 100][train_df['Efficiency'] >= 35].count() / train_df['Efficiency'].count()

0.9899810452206877

**What is the distribution of categorical features?**

- `SugarScale` has two possible values with **97%** being `Specific Gravity` (top=`Specific Gravity`, freq=28,796/count=29,544)
- `BrewMethod` has 4 possible values with **67%** being `All Grain` (top=`All Grain`, freq=19934/count=29544)
- `PrimmingMethod` has only **9%** non-null values (count=2766/total=29544).
- `PrimmingAmmount` has only **7%** non-null values (count=1973/total=29544).

In [15]:
train_df.describe(include=['O'])

,SugarScale,BrewMethod,PrimingMethod,PrimingAmount
count,29544,29544,2766,1973
unique,2,4,480,1016
top,Specific Gravity,All Grain,Corn Sugar,5 oz
freq,28796,19934,277,94


## Cleanning up nonsense values (public entries data)

In [16]:
clean_train_df = train_df.copy()

clean_train_df = clean_train_df[clean_train_df['OG'] <= 1.15][clean_train_df['OG'] >= 1.005]
clean_train_df = clean_train_df[clean_train_df['FG'] <= 1.05][clean_train_df['FG'] >= 1.001]
clean_train_df = clean_train_df[clean_train_df['ABV'] <= 20][clean_train_df['ABV'] >= 0.1]
clean_train_df = clean_train_df[clean_train_df['IBU'] <= 150][clean_train_df['IBU'] >= 0.1]
clean_train_df = clean_train_df[clean_train_df['Color'] <= 45][clean_train_df['Color'] >= 0]
clean_train_df = clean_train_df[clean_train_df['BoilGravity'] <= 1.15][clean_train_df['BoilGravity'] >= 1.005]
clean_train_df = clean_train_df[clean_train_df['Efficiency'] <= 100][clean_train_df['Efficiency'] >= 35]
clean_train_df = clean_train_df[clean_train_df['SugarScale'] == 'Specific Gravity']

clean_train_df['BeerID'].count() / train_df['BeerID'].count()

0.8362103980503656

In [17]:
clean_test_df = test_df.copy()

clean_test_df = clean_test_df[clean_test_df['OG'] <= 1.15][clean_test_df['OG'] >= 1.005]
clean_test_df = clean_test_df[clean_test_df['FG'] <= 1.05][clean_test_df['FG'] >= 1.001]
clean_test_df = clean_test_df[clean_test_df['ABV'] <= 20][clean_test_df['ABV'] >= 0.1]
clean_test_df = clean_test_df[clean_test_df['IBU'] <= 150][clean_test_df['IBU'] >= 0.1]
clean_test_df = clean_test_df[clean_test_df['Color'] <= 45][clean_test_df['Color'] >= 0]
clean_test_df = clean_test_df[clean_test_df['BoilGravity'] <= 1.15][clean_test_df['BoilGravity'] >= 1.005]
clean_test_df = clean_test_df[clean_test_df['Efficiency'] <= 100][clean_test_df['Efficiency'] >= 35]
clean_test_df = clean_test_df[clean_test_df['SugarScale'] == 'Specific Gravity']

clean_test_df['BeerID'].count() / test_df['BeerID'].count()

0.8381397238017871

### Assumtions based on data analysis

We arrive at following assumptions based on data analysis done so far. We may validate these assumptions further before taking appropriate actions.

**Correlating.**

We want to know how well does each feature correlate with Survival. We want to do this early in our project and match these quick correlations with modelled correlations later in the project.

**Completing.**

1. We may want to complete Age feature as it is definitely correlated to survival.
2. We may want to complete the Embarked feature as it may also correlate with survival or another important feature.

**Correcting.**

1. Ticket feature may be dropped from our analysis as it contains high ratio of duplicates (22%) and there may not be a correlation between Ticket and survival.
2. Cabin feature may be dropped as it is highly incomplete or contains many null values both in training and test dataset.
3. PassengerId may be dropped from training dataset as it does not contribute to survival.
4. Name feature is relatively non-standard, may not contribute directly to survival, so maybe dropped.

**Creating.**

1. We may want to create a new feature called Family based on Parch and SibSp to get total count of family members on board.
2. We may want to engineer the Name feature to extract Title as a new feature.
3. We may want to create new feature for Age bands. This turns a continous numerical feature into an ordinal categorical feature.
4. We may also want to create a Fare range feature if it helps our analysis.

**Classifying.**

We may also add to our assumptions based on the problem description noted earlier.

1. Women (Sex=female) were more likely to have survived.
2. Children (Age<?) were more likely to have survived. 
3. The upper-class passengers (Pclass=1) were more likely to have survived.

## Analyze by pivoting features

To confirm some of our observations and assumptions, we can quickly analyze our feature correlations by pivoting features against each other. We can only do so at this stage for features which do not have any empty values. It also makes sense doing so only for features which are categorical (Sex), ordinal (Pclass) or discrete (SibSp, Parch) type.

- **Pclass** We observe significant correlation (>0.5) among Pclass=1 and Survived (classifying #3). We decide to include this feature in our model.
- **Sex** We confirm the observation during problem definition that Sex=female had very high survival rate at 74% (classifying #1).
- **SibSp and Parch** These features have zero correlation for certain values. It may be best to derive a feature or a set of features from these individual features (creating #1).

In [18]:
clean_train_df[['OG', 'StyleID']].groupby(['StyleID'], as_index=False).mean().sort_values(by='OG', ascending=False)

,StyleID,OG
110,117,1.106500
60,63,1.103707
4,5,1.101045
161,172,1.098385
123,132,1.093612
82,87,1.090725
19,21,1.090165
157,168,1.088912
150,160,1.085713
151,161,1.085000


In [19]:
clean_train_df[['FG', 'StyleID']].groupby(['StyleID'], as_index=False).mean().sort_values(by='FG', ascending=False)

,StyleID,FG
60,63,1.025478
157,168,1.023316
123,132,1.023176
4,5,1.022655
150,160,1.022333
161,172,1.022154
82,87,1.022150
59,62,1.020222
51,54,1.020139
152,162,1.018942


In [20]:
clean_train_df[["ABV", "StyleID"]].groupby(['StyleID'], as_index=False).mean().sort_values(by='ABV', ascending=False)

,StyleID,ABV
110,117,12.665000
4,5,10.329455
60,63,10.319457
151,161,10.165000
161,172,10.003077
19,21,9.435588
123,132,9.287758
82,87,9.017500
30,32,8.759444
24,26,8.665163


In [21]:
clean_train_df[["Color", "StyleID"]].groupby(['StyleID'], as_index=False).mean().sort_values(by='Color', ascending=False)

,StyleID,Color
123,132,38.233879
82,87,37.179125
68,72,36.686559
152,162,36.354124
11,12,36.215269
86,91,35.429545
88,93,35.062911
107,114,34.750653
155,166,34.570909
55,58,34.247347


In [22]:
clean_train_df[["IBU", "StyleID"]].groupby(['StyleID'], as_index=False).mean().sort_values(by='IBU', ascending=False)

,StyleID,IBU
81,86,96.666021
53,56,91.454596
4,5,90.348545
123,132,74.874182
151,161,73.190000
138,148,72.973351
12,13,71.877955
6,7,67.745488
137,147,67.645250
141,151,66.448952


## Analyze by visualizing data

Now we can continue confirming some of our assumptions using visualizations for analyzing the data.

### Correlating numerical features

Let us start by understanding correlations between numerical features and our solution goal (Survived).

A histogram chart is useful for analyzing continous numerical variables like Age where banding or ranges will help identify useful patterns. The histogram can indicate distribution of samples using automatically defined bins or equally ranged bands. This helps us answer questions relating to specific bands (Did infants have better survival rate?)

Note that x-axis in historgram visualizations represents the count of samples or passengers.

**Observations.**

- Infants (Age <=4) had high survival rate.
- Oldest passengers (Age = 80) survived.
- Large number of 15-25 year olds did not survive.
- Most passengers are in 15-35 age range.

**Decisions.**

This simple analysis confirms our assumptions as decisions for subsequent workflow stages.

- We should consider Age (our assumption classifying #2) in our model training.
- Complete the Age feature for null values (completing #1).
- We should band age groups (creating #3).

In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

### Correlating numerical and ordinal features

We can combine multiple features for identifying correlations using a single plot. This can be done with numerical and categorical features which have numeric values.

**Observations.**

- Pclass=3 had most passengers, however most did not survive. Confirms our classifying assumption #2.
- Infant passengers in Pclass=2 and Pclass=3 mostly survived. Further qualifies our classifying assumption #2.
- Most passengers in Pclass=1 survived. Confirms our classifying assumption #3.
- Pclass varies in terms of Age distribution of passengers.

**Decisions.**

- Consider Pclass for model training.

In [ ]:
# grid = sns.FacetGrid(train_df, col='Pclass', hue='Survived')
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

### Correlating categorical features

Now we can correlate categorical features with our solution goal.

**Observations.**

- Female passengers had much better survival rate than males. Confirms classifying (#1).
- Exception in Embarked=C where males had higher survival rate. This could be a correlation between Pclass and Embarked and in turn Pclass and Survived, not necessarily direct correlation between Embarked and Survived.
- Males had better survival rate in Pclass=3 when compared with Pclass=2 for C and Q ports. Completing (#2).
- Ports of embarkation have varying survival rates for Pclass=3 and among male passengers. Correlating (#1).

**Decisions.**

- Add Sex feature to model training.
- Complete and add Embarked feature to model training.

In [ ]:
# grid = sns.FacetGrid(train_df, col='Embarked')
grid = sns.FacetGrid(train_df, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

### Correlating categorical and numerical features

We may also want to correlate categorical features (with non-numeric values) and numeric features. We can consider correlating Embarked (Categorical non-numeric), Sex (Categorical non-numeric), Fare (Numeric continuous), with Survived (Categorical numeric).

**Observations.**

- Higher fare paying passengers had better survival. Confirms our assumption for creating (#4) fare ranges.
- Port of embarkation correlates with survival rates. Confirms correlating (#1) and completing (#2).

**Decisions.**

- Consider banding Fare feature.

In [ ]:
# grid = sns.FacetGrid(train_df, col='Embarked', hue='Survived', palette={0: 'k', 1: 'w'})
grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', size=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

## Wrangle data

We have collected several assumptions and decisions regarding our datasets and solution requirements. So far we did not have to change a single feature or value to arrive at these. Let us now execute our decisions and assumptions for correcting, creating, and completing goals.

### Correcting by dropping features

This is a good starting goal to execute. By dropping features we are dealing with fewer data points. Speeds up our notebook and eases the analysis.

Based on our assumptions and decisions we want to drop the Cabin (correcting #2) and Ticket (correcting #1) features.

Note that where applicable we perform operations on both training and testing datasets together to stay consistent.

In [ ]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

print("After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

### Creating new feature extracting from existing

We want to analyze if Name feature can be engineered to extract titles and test correlation between titles and survival, before dropping Name and PassengerId features.

In the following code we extract Title feature using regular expressions. The RegEx pattern `(\w+\.)` matches the first word which ends with a dot character within Name feature. The `expand=False` flag returns a DataFrame.

**Observations.**

When we plot Title, Age, and Survived, we note the following observations.

- Most titles band Age groups accurately. For example: Master title has Age mean of 5 years.
- Survival among Title Age bands varies slightly.
- Certain titles mostly survived (Mme, Lady, Sir) or did not (Don, Rev, Jonkheer).

**Decision.**

- We decide to retain the new Title feature for model training.

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

We can replace many titles with a more common name or classify them as `Rare`.

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

We can convert the categorical titles to ordinal.

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

Now we can safely drop the Name feature from training and testing datasets. We also do not need the PassengerId feature in the training dataset.

In [ ]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

### Converting a categorical feature

Now we can convert features which contain strings to numerical values. This is required by most model algorithms. Doing so will also help us in achieving the feature completing goal.

Let us start by converting Sex feature to a new feature called Gender where female=1 and male=0.

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

### Completing a numerical continuous feature

Now we should start estimating and completing features with missing or null values. We will first do this for the Age feature.

We can consider three methods to complete a numerical continuous feature.

1. A simple way is to generate random numbers between mean and [standard deviation](https://en.wikipedia.org/wiki/Standard_deviation).

2. More accurate way of guessing missing values is to use other correlated features. In our case we note correlation among Age, Gender, and Pclass. Guess Age values using [median](https://en.wikipedia.org/wiki/Median) values for Age across sets of Pclass and Gender feature combinations. So, median Age for Pclass=1 and Gender=0, Pclass=1 and Gender=1, and so on...

3. Combine methods 1 and 2. So instead of guessing age values based on median, use random numbers between mean and standard deviation, based on sets of Pclass and Gender combinations.

Method 1 and 3 will introduce random noise into our models. The results from multiple executions might vary. We will prefer method 2.

In [ ]:
# grid = sns.FacetGrid(train_df, col='Pclass', hue='Gender')
grid = sns.FacetGrid(train_df, row='Pclass', col='Sex', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

Let us start by preparing an empty array to contain guessed Age values based on Pclass x Gender combinations.

In [ ]:
guess_ages = np.zeros((2,3))
guess_ages

Now we iterate over Sex (0 or 1) and Pclass (1, 2, 3) to calculate guessed values of Age for the six combinations.

In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

Let us create Age bands and determine correlations with Survived.

In [ ]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

Let us replace Age with ordinals based on these bands.

In [ ]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()

We can now remove the AgeBand feature.

In [ ]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

### Create new feature combining existing features

We can create a new feature for FamilySize which combines Parch and SibSp. This will enable us to drop Parch and SibSp from our datasets.

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

We can create another feature called IsAlone.

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

Let us drop Parch, SibSp, and FamilySize features in favor of IsAlone.

In [ ]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

We can also create an artificial feature combining Pclass and Age.

In [ ]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

### Completing a categorical feature

Embarked feature takes S, Q, C values based on port of embarkation. Our training dataset has two missing values. We simply fill these with the most common occurance.

In [ ]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

### Converting categorical feature to numeric

We can now convert the EmbarkedFill feature by creating a new numeric Port feature.

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

### Quick completing and converting a numeric feature

We can now complete the Fare feature for single missing value in test dataset using mode to get the value that occurs most frequently for this feature. We do this in a single line of code.

Note that we are not creating an intermediate new feature or doing any further analysis for correlation to guess missing feature as we are replacing only a single value. The completion goal achieves desired requirement for model algorithm to operate on non-null values.

We may also want round off the fare to two decimals as it represents currency.

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

We can now create FareBand.

In [ ]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

Convert the Fare feature to ordinal values based on the FareBand.

In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

And the test dataset.

In [ ]:
test_df.head(10)

## Model, predict and solve

Now we are ready to train a model and predict the required solution. There are 60+ predictive modelling algorithms to choose from. We must understand the type of problem and solution requirement to narrow down to a select few models which we can evaluate. Our problem is a classification and regression problem. We want to identify relationship between output (Survived or not) with other variables or features (Gender, Age, Port...). We are also perfoming a category of machine learning which is called supervised learning as we are training our model with a given dataset. With these two criteria - Supervised Learning plus Classification and Regression, we can narrow down our choice of models to a few. These include:

- Logistic Regression
- KNN or k-Nearest Neighbors
- Support Vector Machines
- Naive Bayes classifier
- Decision Tree
- Random Forrest
- Perceptron
- Artificial neural network
- RVM or Relevance Vector Machine

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

Logistic Regression is a useful model to run early in the workflow. Logistic regression measures the relationship between the categorical dependent variable (feature) and one or more independent variables (features) by estimating probabilities using a logistic function, which is the cumulative logistic distribution. Reference [Wikipedia](https://en.wikipedia.org/wiki/Logistic_regression).

Note the confidence score generated by the model based on our training dataset.

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

We can use Logistic Regression to validate our assumptions and decisions for feature creating and completing goals. This can be done by calculating the coefficient of the features in the decision function.

Positive coefficients increase the log-odds of the response (and thus increase the probability), and negative coefficients decrease the log-odds of the response (and thus decrease the probability).

- Sex is highest positivie coefficient, implying as the Sex value increases (male: 0 to female: 1), the probability of Survived=1 increases the most.
- Inversely as Pclass increases, probability of Survived=1 decreases the most.
- This way Age*Class is a good artificial feature to model as it has second highest negative correlation with Survived.
- So is Title as second highest positive correlation.

In [ ]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

Next we model using Support Vector Machines which are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. Given a set of training samples, each marked as belonging to one or the other of **two categories**, an SVM training algorithm builds a model that assigns new test samples to one category or the other, making it a non-probabilistic binary linear classifier. Reference [Wikipedia](https://en.wikipedia.org/wiki/Support_vector_machine).

Note that the model generates a confidence score which is higher than Logistics Regression model.

In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

In pattern recognition, the k-Nearest Neighbors algorithm (or k-NN for short) is a non-parametric method used for classification and regression. A sample is classified by a majority vote of its neighbors, with the sample being assigned to the class most common among its k nearest neighbors (k is a positive integer, typically small). If k = 1, then the object is simply assigned to the class of that single nearest neighbor. Reference [Wikipedia](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm).

KNN confidence score is better than Logistics Regression but worse than SVM.

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

In machine learning, naive Bayes classifiers are a family of simple probabilistic classifiers based on applying Bayes' theorem with strong (naive) independence assumptions between the features. Naive Bayes classifiers are highly scalable, requiring a number of parameters linear in the number of variables (features) in a learning problem. Reference [Wikipedia](https://en.wikipedia.org/wiki/Naive_Bayes_classifier).

The model generated confidence score is the lowest among the models evaluated so far.

In [ ]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

The perceptron is an algorithm for supervised learning of binary classifiers (functions that can decide whether an input, represented by a vector of numbers, belongs to some specific class or not). It is a type of linear classifier, i.e. a classification algorithm that makes its predictions based on a linear predictor function combining a set of weights with the feature vector. The algorithm allows for online learning, in that it processes elements in the training set one at a time. Reference [Wikipedia](https://en.wikipedia.org/wiki/Perceptron).

In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

This model uses a decision tree as a predictive model which maps features (tree branches) to conclusions about the target value (tree leaves). Tree models where the target variable can take a finite set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees. Reference [Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning).

The model confidence score is the highest among models evaluated so far.

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

The next model Random Forests is one of the most popular. Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees (n_estimators=100) at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Reference [Wikipedia](https://en.wikipedia.org/wiki/Random_forest).

The model confidence score is the highest among models evaluated so far. We decide to use this model's output (Y_pred) for creating our competition submission of results.

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

### Model evaluation

We can now rank our evaluation of all the models to choose the best one for our problem. While both Decision Tree and Random Forest score the same, we choose to use Random Forest as they correct for decision trees' habit of overfitting to their training set. 

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('../output/submission.csv', index=False)

Our submission to the competition site Kaggle results in scoring 3,883 of 6,082 competition entries. This result is indicative while the competition is running. This result only accounts for part of the submission dataset. Not bad for our first attempt. Any suggestions to improve our score are most welcome.

## References

This notebook has been created based on great work done solving the Titanic competition and other sources.

- [A journey through Titanic](https://www.kaggle.com/omarelgabry/titanic/a-journey-through-titanic)
- [Getting Started with Pandas: Kaggle's Titanic Competition](https://www.kaggle.com/c/titanic/details/getting-started-with-random-forests)
- [Titanic Best Working Classifier](https://www.kaggle.com/sinakhorami/titanic/titanic-best-working-classifier)